In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import functools
import gym
from Config import Config
# from util import train
from Models import ActorCritic
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Memory import Memory
from baselines.common.cmd_util import make_env
from baselines.common.atari_wrappers import wrap_deepmind, make_atari
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecEnvWrapper
import pdb

from PPO import PPOPixel
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Models import ActorCritic
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import copy
import gym
import torch
import numpy as np
from collections import deque

import matplotlib.pyplot as plt

env_id = "BreakoutNoFrameskip-v4"
env = make_atari(env_id)
env = wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=True, scale=False)

# config = Config(gym.make('CartPole-v1'))
config = Config(env)

config.update_every = 500
config.num_learn = 4
config.win_condition = 230
config.n_episodes = 1000
config.max_t = 700

config.Memory = Memory
config.Model = ActorCritic
config.head_model = functools.partial(cnn_head_model, config)
config.actor_model = functools.partial(actor_model, config)
config.critic_model = functools.partial(critic_model, config)


In [47]:
class ImageToPyTorch(gym.ObservationWrapper):
    """
    Image shape to channels x weight x height
    """

    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(old_shape[-1], old_shape[0], old_shape[1]),
            dtype=np.uint8,
        )

    def observation(self, observation):
        return np.transpose(observation, axes=(2, 0, 1))

def wrap_pytorch(env):
    return ImageToPyTorch(env)

In [48]:
def make_env(gym_id, seed, idx):
    def thunk():
        env = gym.make(gym_id)
       
        env = wrap_pytorch(
            wrap_deepmind(
                env,
                clip_rewards=True,
                frame_stack=True,
                scale=False,
            )
        )
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return thunk

In [49]:
env = make_env(env_id, 123343534, 1)
env = env()

## PPO test

In [50]:
from PPO import PPOPixel
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Models import ActorCritic
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import copy
import gym
import torch
import numpy as np
from collections import deque

# env = copy.deepcopy(config.env)
steps = 0
scores_deque = deque(maxlen=100)
scores = []
average_scores = []
max_score = -np.Inf

agent = PPOPixel(config)

state = env.reset()
score = 0

## Even more broken down

In [54]:
## Costa agent
class Agent(nn.Module):
    def __init__(self, envs, frames=4):
        super(Agent, self).__init__()
        self.network = nn.Sequential(
            Scale(1/255),
            layer_init(nn.Conv2d(frames, 32, 8, stride=4)),
            nn.ReLU(),
            layer_init(nn.Conv2d(32, 64, 4, stride=2)),
            nn.ReLU(),
            layer_init(nn.Conv2d(64, 64, 3, stride=1)),
            nn.ReLU(),
            nn.Flatten(),
            layer_init(nn.Linear(3136, 512)),
            nn.ReLU()
        )
        self.actor = layer_init(nn.Linear(512, envs.action_space.n), std=0.01)
        self.critic = layer_init(nn.Linear(512, 1), std=1)

    def forward(self, x):
        return self.network(x)

    def get_action(self, x, action=None):
        logits = self.actor(self.forward(x))
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy()

    def get_value(self, x):
        return self.critic(self.forward(x))
class Scale(nn.Module):
    def __init__(self, scale):
        super().__init__()
        self.scale = scale

    def forward(self, x):
        return x * self.scale

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

In [55]:
## Original
class ActorCritic(nn.Module):
  """Some Information about ActorCritic"""
  def __init__(self, config):
    super(ActorCritic, self).__init__()

    self.head = config.head_model()

    self.actor = config.actor_model()
    self.actor.add_module(
      "actor_linear",
      nn.Linear(config.hidden_size, config.action_space)
    )

    self.critic = config.critic_model()
    self.critic.add_module(
      "critic_linear",
      nn.Linear(config.hidden_size, 1)
    )

  def forward(self, x):
    x = self.head(x)
    value = self.critic(x)
    action = self.actor(x)
    return action, value
  
  def act(self, x, action=None):
    logits, value = self.forward(x)
    probs = Categorical(logits=logits)
    if action is None:
      action = probs.sample()
    return action, probs.log_prob(action), value, probs.entropy()

In [56]:
from Models import ActorCritic


model = ActorCritic(config)
# Costa
model = Agent(env)

num_steps = 10

obs = torch.zeros((num_steps, 1) + env.observation_space.shape)
actions = torch.zeros(num_steps, 1)
logsprobs = torch.zeros(num_steps, 1)
rewards = torch.zeros(num_steps)
dones = torch.zeros(num_steps)
values = torch.zeros(num_steps)

state = env.reset()

In [63]:


for t in range(num_steps):
    steps += 1

    # Shape obs
    obs[t] = torch.FloatTensor(state)
    
    # Get action & save
    with torch.no_grad():
        # Original
#         logits, value = model(obs[t])
        # costa
        logits = model.actor(model(obs[t]))
        print("Logits: {}".format(logits))
        probs = Categorical(logits=logits)
        action = probs.sample()

    actions[t] = action
    logsprobs[t] = probs.log_prob(action)
    
    # Act logic & save
    next_state, reward, done, _ = env.step(action)
    rewards[t] = reward
    dones[t] = done
    
    state = next_state

    # Update 
    score += reward

print(logsprobs)
print(logsprobs.shape)

Logits: tensor([[-3.6138e-03, -1.0761e-03, -3.0479e-04, -3.6483e-05]])
Logits: tensor([[-3.7068e-03, -9.3832e-04, -2.6323e-04, -2.9061e-05]])
Logits: tensor([[-3.7361e-03, -9.0065e-04, -3.2370e-04, -6.8040e-05]])
Logits: tensor([[-3.6673e-03, -9.8780e-04, -3.8701e-04, -8.8592e-07]])
Logits: tensor([[-3.7360e-03, -9.8863e-04, -3.7301e-04, -2.3885e-05]])
Logits: tensor([[-3.6674e-03, -9.6776e-04, -3.1391e-04,  6.1858e-05]])
Logits: tensor([[-3.6810e-03, -9.2228e-04, -4.1201e-04, -5.6614e-05]])
Logits: tensor([[-0.0037, -0.0009, -0.0004, -0.0002]])
Logits: tensor([[-0.0037, -0.0009, -0.0004, -0.0002]])
Logits: tensor([[-0.0037, -0.0009, -0.0004, -0.0002]])
tensor([[-1.3853],
        [-1.3888],
        [-1.3851],
        [-1.3860],
        [-1.3850],
        [-1.3850],
        [-1.3854],
        [-1.3852],
        [-1.3859],
        [-1.3887]])
torch.Size([10, 1])


In [64]:
b_obs = obs.reshape((-1,)+env.observation_space.shape)
# b_prev_state = obs.squeeze(dim=1)
b_actions = actions.reshape((-1,)+env.action_space.shape)

print("reshaped state space: {}".format(b_obs.shape))
print("reshaped state space: {}".format(b_actions.shape))

reshaped state space: torch.Size([10, 4, 84, 84])
reshaped state space: torch.Size([10])


In [65]:
logits = model.actor(model(b_obs))
logits

tensor([[-3.6138e-03, -1.0761e-03, -3.0479e-04, -3.6482e-05],
        [-3.7068e-03, -9.3832e-04, -2.6323e-04, -2.9062e-05],
        [-3.7361e-03, -9.0065e-04, -3.2370e-04, -6.8039e-05],
        [-3.6673e-03, -9.8780e-04, -3.8701e-04, -8.8499e-07],
        [-3.7360e-03, -9.8863e-04, -3.7301e-04, -2.3887e-05],
        [-3.6674e-03, -9.6776e-04, -3.1391e-04,  6.1858e-05],
        [-3.6810e-03, -9.2228e-04, -4.1201e-04, -5.6616e-05],
        [-3.6554e-03, -8.5339e-04, -4.1674e-04, -2.3131e-04],
        [-3.6731e-03, -8.8651e-04, -4.1540e-04, -1.7479e-04],
        [-3.7117e-03, -9.4507e-04, -3.9467e-04, -1.6706e-04]],
       grad_fn=<AddmmBackward>)

In [67]:
probs = Categorical(logits=logits)
probs = probs.log_prob(b_actions)
probs

tensor([-1.3853, -1.3888, -1.3851, -1.3860, -1.3850, -1.3850, -1.3854, -1.3852,
        -1.3859, -1.3887], grad_fn=<SqueezeBackward1>)